In [1]:
import graphlab as gl
from graphlab import SFrame
from graphlab import SGraph
import numpy as np
from random import sample
import random
import scipy.io as sio
from itertools import izip
from datetime import datetime
from numpy.random import rand
random.seed(123)
np.random.seed(123)

[INFO] This trial license of GraphLab Create is assigned to kanitw@gmail.com and will expire on June 26, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-28333 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1433231156.log
[INFO] GraphLab Server Version: 1.4.0


In [2]:
import lib

In [26]:
reload(lib)
from lib import n, m, ng, nht, user_ids, movie_ids, uids, mids,\
    load, get_features, prefix, remove, get_vertices, get_graph, \
    rmse_u,rmse, dot_feature, getLR, sgd_triple_updater, \
    run_full, run_debug, eta_search

In [ ]:
# import gflags

# FLAGS = gflags.FLAGS
# #gflags.DEFINE_string('train', 'netflix_mm_10000_1000', 'Training File')
# #gflags.DEFINE_string('test', 'netflix_mm_10000_1000', 'Testing File')
# gflags.DEFINE_string('dataset', 'ratings_debug', 'dataset')
# gflags.DEFINE_string('movie', 'movies', 'Testing File')
# gflags.DEFINE_integer('rank', 10, 'Matrix Rank')
# gflags.DEFINE_float('lamb', 0.1, 'Lambda')
# gflags.DEFINE_float('eta', 0.01, 'Learning Rate')
# gflags.DEFINE_integer('maxit', 50, 'Maximum Number of Iterations')
# gflags.DEFINE_integer('rmseint', 5, 'RMSE Computation Interval')
# gflags.DEFINE_integer('cores', -1, 'CPU cores')


In [20]:
movies = sio.mmread('data/movies.mtx').tocsr()
movies.eliminate_zeros()
movies_features = list(get_features(movies[i]) for i in xrange(m))

In [14]:
def sgd_gl_edge(g, X_train, X_test, \
                lambduh, k, eta=0.05, unified=False, lambduh_w=0, Niter=100, \
                e_rmse=0.005, rmse_train=None, test_freq=5):
    get_rmse = rmse_u if unified else rmse
    
    L, R, wu, wm, bu, bm = getLR(g, unified, k)
    rmse_train = [get_rmse(X_train, L, R, wu, wm, bu, bm, movies) if rmse_train is None else rmse_train]
    rmse_test = []
    
    print "%s: %.4f" % (0,rmse_train[-1])
    start = datetime.now()
    
    print 'eta=%s, lambduh=%s, lambduh=%s, unified=%s, lambduh_w=%s' % (eta, lambduh, lambduh, unified, lambduh_w)
    
    for i in xrange(1, Niter+1): 
        g = g.triple_apply(sgd_triple_updater(\
            eta, lambduh, lambduh, unified, lambduh_w), ['factors', 'w','b'])
        
        L, R, wu, wm, bu, bm = getLR(g, unified, k)
        rmse_train.append(get_rmse(X_train, L, R, wu, wm, bu, bm, movies))
        
        print "%s : %.4f (time:%s)" % (i, rmse_train[-1], datetime.now()-start)
        if abs(rmse_train[-1] - rmse_train[-2]) < e_rmse or np.isnan(rmse_train[-1]):
            break
        if test_freq > 0 and i % test_freq == 0:
            rmses_test.append((i,get_rmse(X_test, L, R, wu, wm, bu, bm, movies)))
    
    rmse_test.append((i,get_rmse(X_test, L, R, wu, wm, bu, bm, movies)))
    print "test=%.4f" % (rmse_test[-1][1])
    return rmse_train, rmse_test, L, R, wu, wm, bu, bm
        

In [27]:
# rmse_train, rmse_test, L, R, wu, wm, bu, bm = run_debug()

get_graph 0:00:13.956655
0: 1.7583
eta=0.05, lambduh=0.001, lambduh=0.001, unified=True, lambduh_w=0.001
1 : 1.0050 (time:0:00:31.436064)
test=1.6833


In [8]:
def search_pure_mf(eta=0.05, Niter=20, data='ratings_debug'):
    X_train_debug, X_test_debug = load(data)
    min_rmse_test = float('inf')
    min_k, min_lambduh = None, None
    rmse_map = {}
    for lambduh in [0.001, 0.01, 0.1]:
        for k in [5, 10, 20]:
            g = get_graph(X_train_debug, k, movies)
            rmse_trainunified, rmse_test, L, R, wu, wm, bu, bm = \
                sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, Niter=Niter)
            rmse_map.get(lambduh, {})[k] = rmse_test
            print "l=%s, k=%s, rmse=%.4f" % (lambduh, k, rmse_test[-1][1])
            if rmse_test[-1][1] < min_rmse_test  or np.isnan(rmse_test[-1][1]):
                min_rmse_test = rmse_test[-1][1]
                min_k = k
                min_lambduh = lambduh
    return min_rmse_test, min_lambduh, min_k


def search_pure_mf_coor(eta=0.05, Niter=20, data='ratings_debug'):
    X_train_debug, X_test_debug = load(data)
    min_rmse_test = float('inf')
    min_k, min_lambduh = None, None
    rmse_map = {}    
    lambduh = 0.01
    for k in [5, 10, 20]:
        g = get_graph(X_train_debug, k, movies_features)
        rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
            sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, Niter=Niter)
        rmse_map.get(lambduh, {})[k] = rmse_test
        print "l=%s, k=%s, rmse=%.4f" % (lambduh, k, rmse_test[-1][1])
        if rmse_test[-1][1] < min_rmse_test or np.isnan(rmse_test[-1][1]):
            min_rmse_test = rmse_test[-1][1]
            min_k = k
            min_lambduh = lambduh
    k=min_k
    for lambduh in [0.001, 0.1]:
        g = get_graph(X_train_debug, k, movies_features)
        rmse_trainunified, rmse_test, L, R, wu, wm, bu, bm = \
            sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, Niter=Niter)
        rmse_map.get(lambduh, {})[k] = rmse_test
        print "l=%s, k=%s, rmse=%.4f" % (lambduh, k, rmse_test[-1][1])
        if rmse_test[-1][1] < min_rmse_test  or np.isnan(rmse_test[-1][1]):
            min_rmse_test = rmse_test[-1][1]
            min_lambduh = lambduh
    return min_rmse_test, min_lambduh, min_k
    
def run_pure_mf(min_lambduh, min_k, eta=0.05):
    X_train, X_test = load('ratings')
    g = get_graph(X_train, 5, movies_features)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k)
    print rmse_test[-1][1]
    return rmse_map, rmse_train, rmse_test

In [9]:
# optimal_mf = search_pure_mf()
optimal_mf = search_pure_mf(data='ratings_debug_small', Niter=1)

NameError: global name 'n' is not defined

In [ ]:
# rmse_map, rmse_train, rmse_test = run_pure_mf(min_lambduh, min_k)

In [105]:
def search_cf(eta=0.05, Niter=20, data='ratings_debug'):
    X_train_debug, X_test_debug = load(data)
    min_rmse_test = float('inf')
    min_k, min_lambduh, min_lambduh_w = None, None, None
    rmse_map = {}
    lambduh, lambduh_w = 0.01, 0.01
    for k in [5, 10, 20]:
        g = get_graph(X_train_debug, k, movies_features)
        rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
            sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, \
                        unified=True, lambduh_w=lambduh_w, Niter=Niter)
        rmse_map.get(lambduh, {}).get(k,{})[lambduh_w] = rmse_test
        print "l=%s, k=%s, l_w=%s, rmse=%.4f" % (lambduh, k, lambduh_w, rmse_test[-1][1])
        if rmse_test[-1][1] < min_rmse_test or np.isnan(rmse_test[-1][1]):
            min_rmse_test = rmse_test[-1][1]
            min_k = k
    k=min_k
    for lambduh in [0.001, 0.01, 0.1]:
        for lambduh_w in [0.001, 0.01, 0.1]:
            g = get_graph(X_train_debug, k, movies_features)
            rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
                sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, \
                            unified=True, lambduh_w=lambduh_w, Niter=Niter)
            rmse_map.get(lambduh, {}).get(k,{})[lambduh_w] = rmse_test
            
            print "l=%s, k=%s, l_w=%s, rmse=%.4f" % (lambduh, k, lambduh_w, rmse_test[-1][1])
            if rmse_test[-1][1] < min_rmse_test or np.isnan(rmse_test[-1][1]):
                min_rmse_test = rmse_test[-1][1]
                min_lambduh = lambduh
                min_lambduh_w = lambduh_w

    return min_rmse_test, min_lambduh, min_k, min_lambduh_w
    
def run_cf(min_lambduh, min_k, min_lambduh_w, eta=0.05):
    X_train, X_test = load('ratings')
    g = get_graph(X_train, min_k, movies_features)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k, min_eta, \
                                unified=True, lambduh_w=min_lambduh_w)
    print rmse_test[-1][1]
    return rmse_map, rmse_train, rmse_test

In [ ]:
# def search_cf(eta=0.05):
#     X_train_debug, X_test_debug = load('ratings_debug')
#     min_rmse_test = float('inf')
#     min_k, min_lambduh, min_lambduh_w = None, None, None
#     rmse_map = {}
#     for k in [5, 10, 20]:
#         for lambduh in [0.001, 0.01, 0.1]:
#             for lambduh_w in [0.001, 0.01, 0.1]:
#                 g = get_graph(X_train_debug, k, movies_features)
#                 rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
#                     sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, \
#                                 unified=True, lambduh_w=lambduh_w)
#                 rmse_map.get(lambduh, {}).get(k,{})[lambduh_w] = rmse_test
#                 print "l=%s, k=%s, l_w=%s, rmse=%.4f" % (lambduh, k, lambduh_w, rmse_test)
#                 if rmse_test < min_rmse_test or np.isnan(rmse_test):
#                     min_rmse_test = rmse_test
#                     min_k = k
#                     min_lambduh = lambduh
#                     min_lambduh_w = lambduh_w

#     return min_rmse_test, min_lambduh, min_k, min_lambduh_w

In [ ]:
# run coldstart to predict cold-start dataset
def run_cf2(min_lambduh, min_k, min_lambduh_w, eta=0.05):
    X_train, X_test = load('ratings_cs')
    g = get_graph(X_train, min_k, movies_features)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k, min_eta, \
                                unified=true, lambduh_w=min_lambduh_w)
    print rmse_test
    return rmse_map, rmse_train, rmse_test

In [ ]:
# optimal_cf = search_cf()
optimal_cf = search_cf(data='ratings_debug_small', Niter=1)

In [ ]:
# rmses_cf = run_cf(min_lambduh, min_k, min_lambduh_w)
# rmses_cf2 = run_cf2(min_lambduh, min_k, min_lambduh_w)